In [1]:
import numpy as np
from numba import cuda, float64
import cupy as cp
import math
import matplotlib.pyplot as plt

In [103]:
def solve(S,I,lamb,tstep=.1,t_max = 50000):

    Ly,Lx = S.shape
    threadsperblock = (32,32)
    blockspergrid = (math.ceil(Lx/32),math.ceil(Ly/32))
   
    @cuda.jit()
    def forces(S,I,lamb,fS,fI):
        i,j = cuda.grid(2)
        if i < Ly and j < Lx:
            fS[j,i] = -S[j,i]*I[j,i] + S[(j+1)%Lx,i] + S[(j-1+Lx)%Lx,i] + S[j,(i+1)%Ly] + S[j,(i-1+Ly)%Ly] - 4*S[j,i]
            fI[j,i] = S[j,i]*I[j,i] - lamb[j,i]*I[j,i] + I[(j+1)%Lx,i] + I[(j-1+Lx)%Lx,i] + I[j,(i+1)%Ly] + I[j,(i-1+Ly)%Ly] - 4*I[j,i]
    
    fS = cp.zeros_like(S)
    fI = cp.zeros_like(I)

    for _ in range(t_max):
        forces[blockspergrid,threadsperblock](S,I,lamb,fS,fI)
        S += tstep*fS
        I += tstep*fI
        S[:,0] = S[:,-1] = I[:,0] = I[:,-1] = 0

    return S,I

def solve2(S,I,lamb,tstep=.1,t_max = 50000):
    for _ in range(t_max):
        S += tstep*(-S*I + laplacian(S))
        I += tstep*(S*I - lamb*I + laplacian(I))
        S[:,0] = S[:,-1] = I[:,0] = I[:,-1] = 0
    return S,I

def laplacian(grid):
    return cp.roll(grid,1,axis=0) + cp.roll(grid,-1,axis=0) + cp.roll(grid,1,axis=1) + cp.roll(grid,-1,axis=1) - 4*grid

In [105]:
L = 1024
p = 0
l = .2
lamb = cp.random.choice([0,l],size=(L,L),p = [p,1-p])
S = cp.ones((L,L))
I = cp.zeros((L,L))
S[:,1] = cp.zeros_like(S[:,1])
I[:,1] = cp.ones_like(I[:,1])

b = solve(S,I,lamb,t_max=1000)[1]

In [102]:
L = 1024
p = 0
l = .2
lamb = cp.random.choice([0,l],size=(L,L),p = [p,1-p])
S = cp.ones((L,L))
I = cp.zeros((L,L))
S[:,1] = cp.zeros_like(S[:,1])
I[:,1] = cp.ones_like(I[:,1])

b = solve2(S,I,lamb,t_max=1000)[1]